In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import timeit

In [28]:
def scrape(mainUrl):
    fromDates = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
    toDates = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
    pomUrl = mainUrl + "ALK"
    response = requests.get(pomUrl)
    soup = BeautifulSoup(response.text, 'html.parser')
    sifri = soup.select("#Code > option")
    companies = []
    for s in sifri:
      if not any(char.isdigit() for char in s.text):
        companies.append(s.text)
    dataFrameList = []
    for j in range(len(companies)):
        myList = []
        for i in range(len(fromDates)):
            url = mainUrl + companies[j] + "?FromDate=10.11." + fromDates[i] + "&ToDate=09.11." + toDates[i]
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.select("#resultsTable > tbody > tr")
            for row in rows:
                datum = row.select_one("td:nth-child(1)").text
                cena_posledna = row.select_one("td:nth-child(2)").text
                mak = row.select_one("td:nth-child(3)").text
                minimum = row.select_one("td:nth-child(4)").text
                cena_prosecna = row.select_one("td:nth-child(5)").text
                procent_promet = row.select_one("td:nth-child(6)").text
                kolicina = row.select_one("td:nth-child(7)").text
                BEST_MKD = row.select_one("td:nth-child(8)").text
                vk_promet = row.select_one("td:nth-child(9)").text
        
                cena_dict = {
                    "Company Name": companies[j],
                    "Date": datum,
                    "Price": cena_posledna,
                    "Maximum": mak,
                    "Minimum": minimum,
                    "Average Price": cena_prosecna,
                    "Procent of turnover": procent_promet,
                    "Quantity": kolicina,
                    "BEST MKD": BEST_MKD,
                    "Total turnover": vk_promet
                }
                myList.append(cena_dict)
                
        df = pd.DataFrame(myList)
        dataFrameList.append(df)
            
    return dataFrameList

In [30]:
dataFor1Year = scrape("https://www.mse.mk/mk/stats/symbolhistory/")

In [32]:
concatinatedDF = pd.concat(dataFor1Year)
concatinatedDF.to_csv('DataForCompanies.csv', index=False)